In [1]:
#Importing all necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [32]:
#Acquiring the "SHARK" dataset from my local directory
data = pd.read_csv('/Users/apple/Desktop/GSAF5.csv', encoding = "cp1252")

In [3]:
#Removing all unnecessary features (Columns) that wont affect my output
data_new=data.drop(columns=['Investigator or Source','pdf','href formula','href','original order','Time','Unnamed: 22','Unnamed: 23','Name','Sex ','Injury','Type'], axis=1)

In [4]:
#Adding last 2 columns (Case Number 1 and Case Number 2) to the beggining (Used this method to benefit from the renaming too)
x0=data_new['Case Number']
x1=data_new['Case Number.1']
x2=data_new['Case Number.2']
data_new=data_new.drop(columns=['Case Number'], axis=1)
data_new=data_new.drop(columns=['Case Number.1'], axis=1)
data_new=data_new.drop(columns=['Case Number.2'], axis=1)
data_new.insert(0,'Case_Number',x0)
data_new.insert(1,'Case_Number_1',x1)
data_new.insert(2,'Case_Number_2',x2)

In [5]:
#Checking missing values and rejecting any columns with more than 80% missing data
null=data_new.isna().sum()*100/data_new.count()
null

Case_Number       0.000000
Case_Number_1     0.000000
Case_Number_2     0.000000
Date              0.000000
Year              0.000000
Country           0.722811
Area              7.191413
Location          9.024745
Activity          9.643184
Age              80.972516
Fatal (Y/N)       0.318098
Species          95.945062
dtype: float64

In [6]:
#Checking features with null percentages > 80% and dropping them
null[null>80]

Age         80.972516
Species     95.945062
dtype: float64

In [7]:
#Droping Columns with null percentages > 80%
cols_drop=null[null>80].index
data_new.drop(cols_drop,axis=1,inplace=True)

In [8]:
#I realized that Activity and Location have close percentages of missing values, hence I went to check if there exists any relationship between them
#I will remove all rows that have both Activity and Location missing
#Calculate the percentage of removal to total null 
#Start of by cleaning the activity column, we identify first its missing elements
act_per_rem=round((data_new.Activity.isna() & data_new.Location.isna()).sum()/(data_new.Activity.isna().sum())*100,2)
act_loc_null=data_new.loc[data_new.Activity.isna() & data_new.Location.isna()].index
act_loc_null
data_new.drop(act_loc_null,axis=0,inplace=True)
#15% of 9% for Activity Column removed and 16.5% of 9% for Location Column removed

In [9]:
#Clean the Location Column -> Area Column -> Country Column
#Remove everything that doesnt have both a Location and an Area
loc_per_rem=round((data_new.Location.isna() & data_new.Area.isna()).sum()/(data_new.Location.isna().sum())*100,2)
loc_perm_null=data_new.loc[data_new.Location.isna() & data_new.Area.isna()].index
loc_perm_null
data_new.drop(loc_perm_null,axis=0,inplace=True)
#40% of 7.5% for Location Removed

#Now, I will equate any location that has a 'nan' to 0
data_new['Location']=data_new['Location'].fillna(0)

In [10]:
#For any missing Area, if we have another instance that has same location and a valid Area -> Replace missing Area with the Valid one
loc_not_0=data_new[(data_new.Location!=0)]
    
dct={}    
for i,j in loc_not_0[['Area','Location']].groupby('Location'):
    dct[i]=list(j.Area.unique())

dct={i:[k for k in j if str(k)!='nan'] for i,j in dct.items() if (np.nan in j)&(len(j)>1)}
dct

{'Carlisle Bay': ['Southwest coast'],
 'Carter Cay': ['Florida'],
 'Guadalcanal': ['Guadalcanal Province'],
 'Naples': ['Florida'],
 'Off Manila': ['Mindoro'],
 'Off Tripoli': ['Mediterranean Sea'],
 'Port Moresby': ['Central Province'],
 'Port Said': ['Mediterranean Sea'],
 'Pula': ['Istria', 'Istria County'],
 'Santa Cruz': ['Galapagos Islands'],
 'Savona': ['Liguria'],
 'Shatt-al Arab River': ['Basrah'],
 'Stewart Island': ['Foveaux Strait'],
 'Tiger Beach': ['West End'],
 'Trieste': ['Adriatic Sea']}

In [11]:
#Example for the below Commands
data_new.loc[data_new.Location=='Savona'].Area

4481    Liguria
5043        NaN
Name: Area, dtype: object

In [12]:
#Looping over the Dictionary to match the above condition
for i,j in dct.items():
    data_new.loc[(data_new.Location==i)&(data_new.Area.isna()),'Area']=j[0]

In [13]:
data_new.loc[data_new.Location=='Savona'].Area

4481    Liguria
5043    Liguria
Name: Area, dtype: object

In [14]:
#For the rest of the Null Areas, we will equate them to 0
data_new['Area']=data_new['Area'].fillna(0)

In [15]:
#Now we are doing the same exercise above to Country and Area Columns
null_country_area=data_new[(data_new.Country.isna()==True) & ((data_new.Area!=0))]
null_country_loc=data_new[(data_new.Country.isna()==True) & ((data_new.Location!=0))]
null_count_area=null_country_area[(null_country_area.Area.duplicated())]['Area']
null_count_loc=null_country_loc[(null_country_loc.Activity.duplicated())]['Location']

lst1=[i for i in null_count_area]

data_new[data_new['Area']=='Caribbean Sea']
data_new[data_new['Area']=='Ionian Sea']

#Since no matching is possible with the available info, we will equate all NaN to 0
data_new['Country']=data_new['Country'].fillna(0)

In [16]:
#Try to replace missing values of Activities by taking the mode max # of activities for the respective location

In [17]:
#Re-arrange Activities into smaller buckets (Maybe 5)
act_new = []
for a in data_new.Activity:
    if re.search(r'Surf[\w\s\,]+|surf[\w\s\,]+|[\w\s\,]+surf[\w\s\,]+', str(a)):
        a = 'Surfing'
    elif re.search(r'Fish[\w\s\,]+|fish[\w\s\,]+|[\w\s\,]+fish[\w\s\,]+', str(a)):
        a = 'Fishing'
    elif re.search(r'Spear[\w\s\,]+|spear[\w\s\,]+|[\w\s\,]+spear[\w\s\,]+', str(a)):
        a = 'Fishing'
    elif re.search(r'Swim[\w\s\,]+|swim[\w\s\,]+|[\w\s\,]+swim[\w\s\,]+', str(a)):
        a = 'Swimming'
    elif re.search(r'Div[\w\s\,]+|div[\w\s\,]+|[\w\s\,]+div[\w\s\,]+', str(a)):
        a = 'Diving'
    else: 
        a = 'Others'
    act_new.append(a)
    
data_new=data_new.drop(columns=['Activity'],axis=1)
data_new['Activity_New']=act_new
data_new

,Case_Number,Case_Number_1,Case_Number_2,Date,Year,Country,Area,Location,Fatal (Y/N),Activity_New
0,2016.09.18.c,2016.09.18.c,2016.09.18.c,18-Sep-16,2016,USA,Florida,"New Smyrna Beach, Volusia County",N,Surfing
1,2016.09.18.b,2016.09.18.b,2016.09.18.b,18-Sep-16,2016,USA,Florida,"New Smyrna Beach, Volusia County",N,Surfing
2,2016.09.18.a,2016.09.18.a,2016.09.18.a,18-Sep-16,2016,USA,Florida,"New Smyrna Beach, Volusia County",N,Surfing
3,2016.09.17,2016.09.17,2016.09.17,17-Sep-16,2016,AUSTRALIA,Victoria,Thirteenth Beach,N,Surfing
4,2016.09.15,2016.09.16,2016.09.15,16-Sep-16,2016,AUSTRALIA,Victoria,Bells Beach,N,Surfing
...,...,...,...,...,...,...,...,...,...,...
5987,ND.0005,ND.0005,ND.0005,Before 1903,0,AUSTRALIA,Western Australia,Roebuck Bay,Y,Diving
5988,ND.0004,ND.0004,ND.0004,Before 1903,0,AUSTRALIA,Western Australia,0,Y,Diving
5989,ND.0003,ND.0003,ND.0003,1900-1905,0,USA,North Carolina,Ocracoke Inlet,Y,Swimming
5990,ND.0002,ND.0002,ND.0002,1883-1889,0,PANAMA,0,"Panama Bay 8ºN, 79ºW",Y,Others


In [18]:
#More than 65% within the categories formed!
data_new.Activity_New.value_counts()

Others      1993
Surfing     1160
Fishing     1095
Swimming    1025
Diving       469
Name: Activity_New, dtype: int64

In [19]:
#More than 75% of Countries are within the first 5 categories
data_new.Country.value_counts()

USA                             2109
AUSTRALIA                       1260
SOUTH AFRICA                     559
PAPUA NEW GUINEA                 131
NEW ZEALAND                      123
                                ... 
GABON                              1
EQUATORIAL GUINEA / CAMEROON       1
MEXICO                             1
NETHERLANDS ANTILLES               1
MALDIVE ISLANDS                    1
Name: Country, Length: 181, dtype: int64

In [20]:
#Sorting countries based on the top 5 and the rest as others, No needed for the regex since data is already clean in this column after inspection
count=[]
for i in data_new.Country:
    if i in ['USA', 'AUSTRALIA','SOUTH AFRICA','PAPA NEW GUINEA','NEW ZEALAND']:
        count.append(i)
    else:
        count.append('Others')

data_new=data_new.drop(columns=['Country'],axis=1)
data_new['Country_New']=count

In [22]:
#Adjusting Date
print(set(data_new['Case_Number']))

{'1987.12.20', '1767.00.00', '1964.06.24', '1994.03.01', '2014.02.07.b', '1960.00.00.g', '2015.11.10', '2009.02.11', '1933.05.24', '2008.10.06', '1975.07.04', '1998.10.01.a', '1903.01.10', '2008.10.22', '1974.08.23', '1947.03.12', '1961.10.00.a', '1885.07.26.c', '2003.04.15', '2006.05.19', '1930.02.20', '2013.03.10.b', '2004.04.04', '1988.03.31', '1995.07.07', '1997.09.16', '1932.12.11.R', '2003.04.21.b', '1961.01.06.b', '1966.04.08', '1995.01.02', '1964.06.02', '2000.03.10', '1877.01.24.R', '1895.04.23', '1906.07.00', '1994.09.21', '2014.06.18.b', '2010.10.09', '2015.03.07', '2008.07.25.b', '1995.07.28.c', '1995.08.01', 'ND.0018', '2001.01.09', '2004.07.27.a', '1964.12.06', '2010.06.27', '2015.01.16', '2010.04.19', '2014.10.11', '1936.08.11', '1996.10.05.', '1995.05.12', '1999.11.00.b', '1872.04.03.R', '2010.12.05', '1925.03.10', '1941.11.27.b', '2006.05.02', '1968.04.11.R', '2000.11.06.a', '1927.05.29', '1961.04.21', '2011.10.02', '1937.09.01', '1997.06.07', '1997.05.31.b', '2014.06.

In [23]:
#Keep only the year from the date
data_new['Date_New']=data_new['Case_Number'].str[:4]

In [24]:
#Displaying all Years
print(set(data_new['Date_New']))

{'1893', '1976', '1700', '1963', '1931', '1989', '1807', '1950', '1971', '1905', '1871', '2005', '1904', '1980', '1988', '1832', '1930', '1822', '1972', '1967', '0005', '0.02', '1862', '1983', '1944', '1755', '1954', '1831', '1896', '1847', '1864', '1733', '1617', '1911', '1749', '1949', '1902', '1901', '1941', '1812', '1816', '1787', '1841', '1738', '1889', '1938', '1909', '1927', '1942', '1788', '1948', '1894', '1885', 'ND-0', '1978', '1543', '2002', '1856', '1880', '1877', '0.04', '1912', '1987', '1951', '1637', 'ND-1', '1863', '1958', '1851', '1975', '1937', '1991', '1872', '1869', '1826', '2011', '1721', '1986', '1923', '1990', '1979', '1995', '1874', '1934', '1804', '1829', '2009', '2013', '1873', '1946', '1903', '1825', '1985', '1908', '1928', '1888', '1964', '1957', '1861', '1878', '2012', '1906', '1642', '1891', '0.07', '1900', '1952', '1844', '1965', '1919', '1925', '1920', '1879', '1969', '1940', '1992', '1852', 'nd-0', '2004', '1970', '2014', '1764', '1827', '1779', '1966',

In [25]:
#Turned unknown dates into 0's so that I can transform the whole column data type into float
lst=['ND.0','0.03','0.02','ND.1','0.07','nd.0','0.04','0005','ND-0','0077','ND-1','nd.0','nd-0']
for i in lst:
    data_new.Date_New.loc[(data_new['Date_New']==i)]=0

#Indexed all rows with Date_New = 0 so I can remove them (103 Rows Only)
x=list(data_new[data_new.Date_New==0].index)
data_new=data_new.drop(x,axis=0)

#Changing the type of data in the Date_New column from object to float
data_new.Date_New=data_new.Date_New.astype(int)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
#Arrange Date_New in Bins
data_new.sort_values(by=['Date_New'])

#Binning
labels=['1500-1550','1550-1600','1600-1650','1650-1700','1700-1750','1750-1800','1800-1850','1850-1900','1900-1950','1950-2000','2000-2020']
cutoffs=(1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2050)
data_new['Bins']=pd.cut(data_new.Date_New,cutoffs,labels=labels)
data_new.Bins.value_counts()

data_new

,Case_Number,Case_Number_1,Case_Number_2,Date,Year,Area,Location,Fatal (Y/N),Activity_New,Country_New,Date_New,Bins
0,2016.09.18.c,2016.09.18.c,2016.09.18.c,18-Sep-16,2016,Florida,"New Smyrna Beach, Volusia County",N,Surfing,USA,2016,2000-2020
1,2016.09.18.b,2016.09.18.b,2016.09.18.b,18-Sep-16,2016,Florida,"New Smyrna Beach, Volusia County",N,Surfing,USA,2016,2000-2020
2,2016.09.18.a,2016.09.18.a,2016.09.18.a,18-Sep-16,2016,Florida,"New Smyrna Beach, Volusia County",N,Surfing,USA,2016,2000-2020
3,2016.09.17,2016.09.17,2016.09.17,17-Sep-16,2016,Victoria,Thirteenth Beach,N,Surfing,AUSTRALIA,2016,2000-2020
4,2016.09.15,2016.09.16,2016.09.15,16-Sep-16,2016,Victoria,Bells Beach,N,Surfing,AUSTRALIA,2016,2000-2020
...,...,...,...,...,...,...,...,...,...,...,...,...
5857,1637.00.00.R,1637.00.00.R,1637.00.00.R,Reported 1637,1637,West Bengal,Hooghly River mouth,UNKNOWN,Others,Others,1637,1600-1650
5858,1617.00.00.R,1617.00.00.R,1617.00.00.R,Reported 1617,1617,West Bengal,Ganges Delta,UNKNOWN,Others,Others,1617,1600-1650
5859,1642.00.00,1642.00.00,1642.00.00,1642,1642,New York,Between Manhattan and The Bronx,Y,Swimming,USA,1642,1600-1650
5860,1595.00.00,1595.00.00,1595.00.00,1595,1595,Kerala,River Cochin,UNKNOWN,Others,Others,1595,1550-1600


In [27]:
#Choosing columns needed for my objective
data_new=data_new[['Date_New','Bins','Country_New','Area','Location','Activity_New']]
data_new

,Date_New,Bins,Country_New,Area,Location,Activity_New
0,2016,2000-2020,USA,Florida,"New Smyrna Beach, Volusia County",Surfing
1,2016,2000-2020,USA,Florida,"New Smyrna Beach, Volusia County",Surfing
2,2016,2000-2020,USA,Florida,"New Smyrna Beach, Volusia County",Surfing
3,2016,2000-2020,AUSTRALIA,Victoria,Thirteenth Beach,Surfing
4,2016,2000-2020,AUSTRALIA,Victoria,Bells Beach,Surfing
...,...,...,...,...,...,...
5857,1637,1600-1650,Others,West Bengal,Hooghly River mouth,Others
5858,1617,1600-1650,Others,West Bengal,Ganges Delta,Others
5859,1642,1600-1650,USA,New York,Between Manhattan and The Bronx,Swimming
5860,1595,1550-1600,Others,Kerala,River Cochin,Others


In [28]:
data_new.loc[data_new['Activity_New']]

KeyError: "None of [Index(['Surfing', 'Surfing', 'Surfing', 'Surfing', 'Surfing', 'Fishing',\n       'Others', 'Swimming', 'Surfing', 'Others',\n       ...\n       'Others', 'Swimming', 'Swimming', 'Others', 'Others', 'Others',\n       'Others', 'Swimming', 'Others', 'Diving'],\n      dtype='object', length=5638)] are in the [index]"

In [ ]:
#Grouping
data_new=data_new.groupby(['Bins'])['Activity_New'].agg(lambda x:x.value_counts().to_list())
data_new=data_new.reset_index()

In [ ]:
#Plotting
sns.set()
fig,ax=plt.subplots(figsize=(15,8))
barchart=sns.barplot(data=data_new, x='Bins', y='Activity_New')
plt.title(f'Max Activity per Bin')